# The Airport dataset
For some simple data cleaning and preparation studies we will use an airport dataset from this [website](https://github.com/ismayc/pnwflights14): 
It contains information about all flights that departed from the two major airports of the Pacific Northwest (PNW), SEA in Seattle and PDX in Portland, in 2014: 162,049 flights in total.

In [1]:
import pandas as pd

flights = pd.read_csv("/fs/scratch/PAS1585/flights/flights.csv")
print(flights.head())

   year  month  day  dep_time  dep_delay  arr_time  arr_delay carrier tailnum  \
0  2014      1    1       1.0       96.0     235.0       70.0      AS  N508AS   
1  2014      1    1       4.0       -6.0     738.0      -23.0      US  N195UW   
2  2014      1    1       8.0       13.0     548.0       -4.0      UA  N37422   
3  2014      1    1      28.0       -2.0     800.0      -23.0      US  N547UW   
4  2014      1    1      34.0       44.0     325.0       43.0      AS  N762AS   

   flight origin dest  air_time  distance  hour  minute  
0     145    PDX  ANC     194.0      1542   0.0     1.0  
1    1830    SEA  CLT     252.0      2279   0.0     4.0  
2    1609    PDX  IAH     201.0      1825   0.0     8.0  
3     466    PDX  CLT     251.0      2282   0.0    28.0  
4     121    SEA  ANC     201.0      1448   0.0    34.0  


# Dealing with missing data
Sometimes you have datasets in which there are subsets of the data which are missing some features. 

How do we find these rows?  If the data is truly missing (meaning there is nothing in the place where it should be)  we can use a tool in pandas to find it: Let's use google: **pandas dataframe find rows with nan**.   ("nan" means "not a number")

The answer looks something like this: df1 = df[df.isnull().any(axis=1)]
This finds all rows in dataframe **df** in which any column in that row has a null, and then returns that as a new dataframe called df1.

So let's try it with our flights dataframe:

In [2]:
flights_nulls = flights[flights.isnull().any(axis=1)]
print(flights_nulls.head(20))

     year  month  day  dep_time  dep_delay  arr_time  arr_delay carrier  \
408  2014      1    1       NaN        NaN       NaN        NaN      AS   
409  2014      1    1       NaN        NaN       NaN        NaN      OO   
410  2014      1    1       NaN        NaN       NaN        NaN      OO   
411  2014      1    1       NaN        NaN       NaN        NaN      OO   
412  2014      1    1       NaN        NaN       NaN        NaN      OO   
413  2014      1    1       NaN        NaN       NaN        NaN      OO   
414  2014      1    1       NaN        NaN       NaN        NaN      OO   
415  2014      1    1       NaN        NaN       NaN        NaN      OO   
416  2014      1    1       NaN        NaN       NaN        NaN      UA   
417  2014      1    1       NaN        NaN       NaN        NaN      UA   
418  2014      1    1       NaN        NaN       NaN        NaN      UA   
421  2014      1    2      14.0       84.0    1356.0        NaN      DL   
428  2014      1    2    

Notice the first column printed out above - the one with 408, then 409, etc.   This is the pandas dataframe **index**.  If you watched the video I pointed the class to, this should be familiar.

We can print out some of these same rows from the **original** dataframe by using this index.   We can do this using "**loc**" or "**iloc**".   The difference being:
1.  loc gets rows (or columns) with particular labels from the index. 
2.  iloc gets rows (or columns) at particular positions in the index (so it only takes integers)

In our original dataframe (flights) the *label* is the same as the *position*.   This is **not** true for the derived dataframe flights_nulls.    
 
 Here is a specific row from the original dataframe:

In [5]:
print("Using loc")
print(flights.loc[408,:])
print()
print("Using iloc")
print(flights.iloc[408][:])

Using loc
year           2014
month             1
day               1
dep_time        NaN
dep_delay       NaN
arr_time        NaN
arr_delay       NaN
carrier          AS
tailnum      N763AS
flight           61
origin          SEA
dest            JNU
air_time        NaN
distance        909
hour            NaN
minute          NaN
Name: 408, dtype: object

Using iloc
year           2014
month             1
day               1
dep_time        NaN
dep_delay       NaN
arr_time        NaN
arr_delay       NaN
carrier          AS
tailnum      N763AS
flight           61
origin          SEA
dest            JNU
air_time        NaN
distance        909
hour            NaN
minute          NaN
Name: 408, dtype: object


Here are rows from the derived dataframe:

In [6]:
print("Using loc")
print(flights_nulls.loc[408,:])    ## this uses the index label
print()
print("Using iloc")
print(flights_nulls.iloc[0][:])    ## this uses the position - the zeroth row

Using loc
year           2014
month             1
day               1
dep_time        NaN
dep_delay       NaN
arr_time        NaN
arr_delay       NaN
carrier          AS
tailnum      N763AS
flight           61
origin          SEA
dest            JNU
air_time        NaN
distance        909
hour            NaN
minute          NaN
Name: 408, dtype: object

Using iloc
year           2014
month             1
day               1
dep_time        NaN
dep_delay       NaN
arr_time        NaN
arr_delay       NaN
carrier          AS
tailnum      N763AS
flight           61
origin          SEA
dest            JNU
air_time        NaN
distance        909
hour            NaN
minute          NaN
Name: 408, dtype: object


Back to dealing with missing data!

Here are some options:

1.  Simply remove rows with missing data.
2.  Replace the missing data with the mean of that column.
3.  Replace the missing data with the mean of columns that are similar to that row. For exmple, we could choose rows that are geographically similar.

It is important to think carefully about the data when choosing which option to use.  It might not make sense to replace missing values with the means of the respective columns.    On the other hand removing every row which has a missing value might remove too much data.

Here is how you would remove rows with missing data:

In [7]:
flights_nonulls = flights.dropna()
print("Length of flights DF:",len(flights))
print("Length of flights with no nulls DF:",len(flights_nonulls))
print("Length of flights with nulls DF:",len(flights_nulls))




Length of flights DF: 162049
Length of flights with no nulls DF: 160748
Length of flights with nulls DF: 1301


Removing rows with no nulls only affects 0.8% of the data, so it is probably the easiest approach.

Just for completeness, lets also look at how to replace null data with the mean: 

**NOTE** This will take some time!

In [8]:
flights.fillna(flights.mean(),inplace=True)
print(flights.loc[408,:])

year            2014
month              1
day                1
dep_time     1278.28
dep_delay    6.13386
arr_time      1482.5
arr_delay    2.24087
carrier           AS
tailnum       N763AS
flight            61
origin           SEA
dest             JNU
air_time     152.589
distance         909
hour         12.4795
minute       30.3322
Name: 408, dtype: object


We see that the row we looked at previously (rownum=408) has missing values filled in with the means from those rows (**be patient!!** this will take some time to finish!):

In [9]:
print("Column means\n",flights.mean())

Column means
 year         2014.000000
month           6.605027
day            15.749890
dep_time     1278.283755
dep_delay       6.133859
arr_time     1482.501282
arr_delay       2.240868
flight       1357.356725
air_time      152.588511
distance     1204.514634
hour           12.479515
minute         30.332244
dtype: float64
